In [1]:
from src.returns_analysis import analyse_returns_characteristics
from src.process_selection import process_selection, recommend_process
from src.var import produce_var_results
from src.processes import Processes
from src.get_data import HistoricalData
from src.util import parse_json
import plotly.graph_objects as go
import numpy as np

In [ ]:
config = parse_json("config.json")

histdata = HistoricalData(config["binance"]["API_KEY"], config["binance"]["API_SECRET"])
df = histdata.getKline("SOLUSDC","1h").dropna()
# histdata.save_data("test_data.csv")

In [3]:
# df = pd.read_csv("test_data.csv").dropna()
# df.head()

In [4]:
stats_summary = analyse_returns_characteristics(df)

BASIC STATS
mean : 8.781827899586981e-05
std : 0.06229493066890635
skewness : 5.943754566533116
kurtosis : 107.85390217945759
jb_pvalue : 0.0
min : -0.2784461956867913
max : 1.1074802155377697


In [5]:
test_results = process_selection(df)

In [6]:
recommend_process(test_results,stats_summary)

[{'process': 'jump diffusion (merton)',
  'confidence': 'medium',
  'reason': 'significant jump frequency'},
 {'process': 'Ornstein-Uhlenbeck',
  'confidence': 'medium',
  'reason': 'evidence of mean reversion in returns'},
 {'process': 'heston stochastic volatility',
  'confidence': 'medium',
  'reason': 'non-normal distrib with fat tails and skewness'}]

In [7]:
processes = Processes(df)

GBM

In [8]:
log_returns = processes.gbm_log_returns(60,5)
fig = go.Figure()
for col in log_returns.columns:
    fig.add_trace(
        go.Scatter(
            x = log_returns.index,
            y = log_returns[col],
            name = col,
            mode = "lines"
        )
    )
fig.show()

VaR

In [9]:
price_path = processes.gbm_price_path(24*60,1_000)
produce_var_results(price_path,df)